<a href="https://colab.research.google.com/github/hailusong/nlp-qa/blob/master/nlp_qa_albert_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
1. [AllenNLP Q&A with heatmap](https://demo.allennlp.org/reading-comprehension/MjE0MTE2MQ==)

## Environment Setup
To get the source code of transformer release, do this (assuming the release version is 2.11.0)
```
git fetch -a --tags
git checkout tags/v2.11.0
```

Make sure we have access to file command in Linux

In [2]:
!uname -a
!apt-get install file

Linux 948adb98e3a7 4.19.104+ #1 SMP Wed Feb 19 05:26:34 PST 2020 x86_64 x86_64 x86_64 GNU/Linux
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (236 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 144467 files and directories currently installed.)
Prepa

In [3]:
try:
    import transformers
except ImportError as e:
    # install huggingface
    !pip install transformers

     |████████████████████████████████| 675kB 4.7MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 3.8MB 25.3MB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=a72e720e2be4b6547cf591b9ee5c9be194d701d7a4010f3d49d0a05cbf945fdf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
import transformers

print(torch.__version__)
print(transformers.__version__)

1.5.0+cu101
2.11.0


In [5]:
!ls -al
!rm -rf nlp-qa
!git clone --depth 1 https://github.com/hailusong/nlp-qa.git

total 16
drwxr-xr-x 1 root root 4096 May 29 18:19 .
drwxr-xr-x 1 root root 4096 Jun  4 18:40 ..
drwxr-xr-x 1 root root 4096 Jun  2 16:14 .config
drwxr-xr-x 1 root root 4096 May 29 18:19 sample_data
Cloning into 'nlp-qa'...
fatal: could not read Username for 'https://github.com': No such device or address


In [6]:
!pip install -r nlp-qa/question-answering/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'nlp-qa/question-answering/requirements.txt'


## Configuratiuon

In [0]:
MULTILABEL_INTENT=True
QA_EXTRACTION=False

## Download

### Pre-trained ALBERT v2 language model 
https://huggingface.co/transformers/pretrained_models.html

In [9]:
if MULTILABEL_INTENT:
  from transformers import AutoTokenizer, AutoModelForQuestionAnswering

  import torch
  model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'albert-large-v2')    # Download model and configuration from S3 and cache.
  # model = torch.hub.load('huggingface/pytorch-transformers', 'model', './test/bert_model/')  # E.g. model was saved using `save_pretrained('./test/saved_model/')`
  # model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'albert-large-v2', output_attentions=True)  # Update configuration during loading
  # assert model.config.output_attentions == True
  model


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master


AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=1024, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0, inplace=False)
        

### Pre-trained ALBERT v2 model on SQuAD2
https://huggingface.co/elgeish/cs224n-squad2.0-albert-xxlarge-v1, including:
- config.json, 721.0B
- pytorch_model.bin, 849.2MB
- special_tokens_map.json, 156.0B
- spiece.model, 742.5KB
- tokenizer_config.json, 39.0B

In [0]:
if QA_EXTRACTION:
  from transformers import ( AutoTokenizer, AutoModelForQuestionAnswering )

  tokenizer = AutoTokenizer.from_pretrained("elgeish/cs224n-squad2.0-albert-xxlarge-v1")
  model = AutoModelForQuestionAnswering.from_pretrained("elgeish/cs224n-squad2.0-albert-xxlarge-v1")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [0]:
if QA_EXTRACTION:
  # inspect tokenizer max length
  print(f'Tokenizer max input length {tokenizer.model_max_length}')

Tokenizer max input length 512


## Inference
Source: https://pytorch.org/hub/huggingface_pytorch-transformers/


In [0]:
if QA_EXTRACTION:
  # model = torch.hub.load('huggingface/pytorch-transformers', 'modelForQuestionAnswering', 'bert-large-uncased-whole-word-masking-finetuned-squad')
  # tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-large-uncased-whole-word-masking-finetuned-squad')

  # The format is paragraph first and then question
  # fact = "Jim Henson was a puppeteer"
  # q = "Who was Jim Henson ?"
  fact = input('Fact: ')
  q = input('Question: ')

  # Converts a list of string to a sequence of ids (integer), using the tokenizer and vocabulary. 
  # Also request to add model-specific special tokens (such as beginning of sequence, end of sequence, sequence separator).
  indexed_tokens_fact = tokenizer.encode(fact, add_special_tokens=True)
  indexed_tokens_q = tokenizer.encode(q, add_special_tokens=True)

  # merge fact tokens and Q tokens by removing the START token in Q tokens
  indexed_tokens = indexed_tokens_fact + indexed_tokens_q[1:]
  segments_ids = [0] * len(indexed_tokens_fact) + [1] * len(indexed_tokens_q[1:])

  print(f'Length is {len(indexed_tokens)}, Indexed token: {indexed_tokens}')

  # Predict the start and end positions logits
  segments_tensors = torch.tensor([segments_ids])
  tokens_tensor = torch.tensor([indexed_tokens])

  with torch.no_grad():
      start_logits, end_logits = model(tokens_tensor, token_type_ids=segments_tensors)

  # get the highest prediction
  answer = tokenizer.decode(indexed_tokens[torch.argmax(start_logits):torch.argmax(end_logits)+1])
  print(f'The answer is: {answer}')

  # Or get the total loss which is the sum of the CrossEntropy loss for the start and end token positions (set model to train mode before if used for training)
  # start_positions, end_positions = torch.tensor([12]), torch.tensor([14])
  # multiple_choice_loss = model(tokens_tensor, token_type_ids=segments_tensors, start_positions=start_positions, end_positions=end_positions)

Fact: Robotics is an interdisciplinary branch of engineering and science that includes mechanical engineering, electrical engineering, computer science, and others. Robotics deals with the design, construction, operation, and use of robots, as well as computer systems for their control, sensory feedback, and information processing. These technologies are used to develop machines that can substitute for humans. Robots can be used in any situation and for any purpose, but today many are used in dangerous environments (including bomb detection and de-activation), manufacturing processes, or where humans cannot survive. Robots can take on any form but some are made to resemble humans in appearance. This is said to help in the acceptance of a robot in certain replicative behaviors usually performed by people. Such robots attempt to replicate walking, lifting, speech, cognition, and basically anything a human can do.
Question: What do robots that resemble humans plan  to do?
Length is 183, I